In [166]:
import pymongo                                                                                                                                                                                                                               
from pymongo import MongoClient                                                                                                                                                                                                              
from pprint import pprint#, pformat, PrettyPrinter                                                                                                                                                                                            
# from bson.son import SON                                                                                                                                                                                                                     
from bson.objectid import ObjectId                                                                                                                                                                                                           
# from os.path import exists                                                                                                                                                                                                                   
# import json                                                                                                                                                                                                                                  
# from ipy_table import *        

from mongotools import *

In [167]:
def formatRow(row):
    formats = { 2: u"{}: {}", 3: u"{}: {} ({})" }
    row_format = formats.get( len(row), u"{} " * len(row) )
    return (row_format.format(*row.values()))

In [168]:
# Connect to database and create variables for the collections                                                                                                                                                                                 
client = MongoClient()                                                                                                                                                                                                                       
db = client.lobbyradar                                                                                                                                                                                                                       
entities, relations = db.entities, db.relations

print("%s Entities and %s Relations loaded." % (entities.count(), relations.count())) 

26380 Entities and 32137 Relations loaded.


In [169]:
etools = MongoHelper(entities)
rtools = MongoHelper(relations)

# Ontologie
Um eine Ontologie zu bauen müssen wir uns auf Klasse und Eigenschaften festlegen. Es geht nun also darum, die Daten gut zu untersuchen und die wichtigsten Klassen daraus abzuleiten.

Grundlegend geht es in diesem Datensatz um Vernetzung, weshalb die "Verbindung" so gut wie allen Relationen zu Grunde liegt.

Auf Seiten der Relationen sind die Felder
 - Tags
 - data.desc
 - data.format
 - __data.value.position__
 - data.value.type
 - __data.value.desc__
 - __data.value.amount__
 - __data.value.activity__

besonders aussagekräftig und bieten viele Daten.

### Tags
Die Tags bieten relativ viel Informationen, dafür leider unstrukturiert. Da es viele wenig genutzte Tags gibt, schränken wir die Suche etwas ein.

In [170]:
# Filter Tags with less than 100 occurences
pipeline = make_pipeline(unwind="tags", group="tags", sort=True, min_count=100)

tags = relations.aggregate(pipeline , cursor={})
printCols( [ formatRow(row) for row in tags ], spacing=20)

TypeError: make_pipeline() takes exactly 1 argument (0 given)

### data.XX.desc  | data.XX.format

In [ ]:
desc_pipeline = make_pipeline(unwind="data", group="data.desc", sort=True, min_count=100)
format_pipeline = make_pipeline(unwind="data", group="data.format", sort=True, min_count=100)
desc = relations.aggregate(desc_pipeline, cursor={})
form = relations.aggregate(format_pipeline, cursor={})

printCols( [ u"{:<60} {:<50}".format(d, f) for (d, f) in zip(desc, form) ], columns=1 )

Hier sehen wir deutlich, dass data.desc und data.format so gut wie deckungsgleich sehen. Es reicht also, wenn wir eines der Felder betrachten. 
Auch data.key ist fast gleichbedeutend damit.

### data.value.position

In [ ]:
position_pipeline = make_pipeline(unwind="data", group="data.value.position", sort=True, min_count=100)
position = relations.aggregate(position_pipeline, cursor={})

printCols( [ formatRow(row) for row in position ], columns=4, spacing=40)

Natürlicherweise sind alle Vorstände auch immer assoziiert mit dem jeweiligen Partner:

In [ ]:
p = [
 {'$unwind': '$data'},
 {'$match': {'data.format': {'$eq': "association"}}},
 {'$group': {'_id': { 'position': '$data.value.position', 'format': '$data.format'}, 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}},
 {'$match': {'count': {'$gt': 100}}} 
]
printCols( [ formatRow(row) for row in relations.aggregate(p) ], columns=2, spacing=90)

Dass eine Spende zusammen mit einer "Position" eintritt ist eher seltener..

In [ ]:
p = [
 {'$unwind': '$data'},
 {'$match': {'data.format': {'$eq': "donation"}}},
 {'$group': {'_id': { 'position': '$data.value.position', 'format': '$data.format'}, 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}},
 {'$match': {'count': {'$gt': 100}}} 
]
printCols( [ formatRow(row) for row in relations.aggregate(p) ], columns=2, spacing=90)

Bei der activity gibt es wieder Personen, die eine Position innehaben.

In [ ]:
p = [
 {'$unwind': '$data'},
 {'$match': {'data.format': {'$eq': "activity"}}},
 {'$group': {'_id': { 'position': '$data.value.position', 'format': '$data.format'}, 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}},
 {'$match': {'count': {'$gt': 100}}} 
]
printCols( [ formatRow(row) for row in relations.aggregate(p) ], columns=2, spacing=90)

### data.value.type

In [ ]:
type_pipeline = make_pipeline(unwind="data", group="data.value.type", sort=True, min_count=50)
types = relations.aggregate(type_pipeline, cursor={})

printCols( [ formatRow(row) for row in types ], columns=2)

In [ ]:
subsidiary = relations.find({ "data.value.type": {"$eq": "subsidiary"} }, {'tags': 1, 'data.value': 1, 'entities': 1}).limit(3)
for r in subsidiary:
    pprint(r)

In [ ]:
def lookup_relation(relation_id):
    relation = relations.find_one({ "_id": relation_id }, {'tags': 1, 'data': 1, 'entities': 1})  
    entity_ids = relation.get('entities')
    pprint(relation.get('data'))
    print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
    print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))
    
def lookup_by_value_desc(value_desc):
    relation = relations.find_one({ "data.value.desc": value_desc }, {'tags': 1, 'data': 1, 'entities': 1})  
    entity_ids = relation.get('entities')
    pprint(relation.get('data'))
    print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
    print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

In [ ]:
entitiy_ids = (ObjectId('54c2a4cafe6a42c82bbab93d'), ObjectId('54bd3ca38b934da0634177da'))
print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

In [ ]:
entitiy_ids = (ObjectId('54c2a4b3fe6a42c82bbaaf7e'),
               ObjectId('54bd3ca18b934da063417539'))
print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

In [ ]:
entitiy_ids = (ObjectId('54c2a4b3fe6a42c82bbaaf7e'),
               ObjectId('54cb842abbde97945a06a06f'))
print(etools.lookup_entity(entitiy_ids[0], {'name': 1}))
print(etools.lookup_entity(entitiy_ids[1], {'name': 1}))

### data.value.desc

In [ ]:
desc_pipeline = make_pipeline(unwind="data", group="data.value.desc", sort=True, min_count=50)
descs = relations.aggregate(desc_pipeline, cursor={})

printCols( [ formatRow(row) for row in descs ], columns=2, spacing=70)

In [ ]:
lookup_by_value_desc("Funktionen in Unternehmen")

In [ ]:
lookup_by_value_desc("Geschäftsverbindung")

In [ ]:
lookup_by_value_desc("Funktionen in Vereinen, Verbänden und Stiftungen")

In [ ]:
lookup_by_value_desc("Funktionen in Körperschaften und Anstalten des öffentlichen Rechts")

In [ ]:
lookup_by_value_desc("Bundestag Spenden von Personen und Beiträge von Mandatsträgern")

### data.value.amount

In [ ]:
amount_pipeline = make_pipeline(unwind="data", group="data.value.amount", sort=True, min_count=50)
amounts = relations.aggregate(amount_pipeline, cursor={})

printCols( [ formatRow(row) for row in amounts ], columns=2)

In [ ]:
amounts = relations.find({ "data.value.amount": {"$exists": 1}}).limit(2)
pprint(list(amounts))

### Beide relation-IDs stammen von einer Parteispende-Beziehung

In [ ]:
pprint(lookup_relation(ObjectId('54c2a4f0fe6a42c82bbabb54')))
pprint(lookup_relation(ObjectId('54c2a4f0fe6a42c82bbabb55')))

In [ ]:
amounts = relations.find({ "data.value.amount": {"$exists": 1}}, {'data.value': 1}).limit(2)
pprint(list(amounts))

### data.value.activity

In [ ]:
activity_pipeline = make_pipeline(unwind="data", group="data.value.activity", sort=True)
activities = relations.aggregate(activity_pipeline, cursor={})

printCols( [ formatRow(row) for row in activities ], columns=3, spacing=70)

In [ ]:
pprint(list(relations.find({ "data.value.activity": "Vortrag" }).limit(3)))